# LLM From Scratch
#### (By: Mark Ehab Aziz)
##### (Built under Python 3.11.4)

Graduation Project for ***Sprints***, to build a LLM (Large Language Model) from scratch without the use of any Transformer related libraries, in order to classify comments scraped from some given website.

Notebook will include:
- List of imported libraries.
- Deep dive into the data.
- Markdown cells to explain the detail of every step, along with reason.
- Custom defined functions and classes.
- Own Transformer from scratch.

List of Libraries/Dependencies:
- [Pandas](https://pandas.pydata.org/docs/index.html)
- [NumPy](https://numpy.org/doc/stable/)
- [NLTK](https://www.nltk.org/)
    - [NLTK Regex Tokenizer](https://www.nltk.org/howto/tokenize.html)
    - [NLTK Snowball Stemmer](https://www.nltk.org/howto/stem.html)
    - [NLTK WordNet Lemmatizer](https://www.nltk.org/howto/wordnet.html)
    - [NLTK English Words (Stopwords too)](https://www.nltk.org/howto/corpus.html)
- [PyTorch (`torch` and `torch.nn`)](https://pytorch.org/docs/stable/index.html)

Note: NLTK will be used later for tokenization using RegEx.

# Library Imports
Importing libraries that will be used in order to implement our own LLM.

In [ ]:
import pandas as pd                             # Pandas for DataFrame manipulation
import numpy as np                              # Linear Algebra and Mathematical Operations
import nltk                                     # Downloading word sets
from nltk.stem import SnowballStemmer           # Stemming
from nltk.stem import WordNetLemmatizer         # Lemmatization (Better word yields)
from nltk.corpus import stopwords               # Stopwords
from nltk.tokenize import regexp_tokenize       # Tokenization using RegEx (Regular Expressions)

In [ ]:
# Downloading external dependencies to be used
# Such as Stopwords, English-only words

# Stopwords
nltk.download('stopwords')

# English Only
nltk.download('words')

# Wordnet for Lemmatizer
nltk.download('wordnet')

# Setting constants for both normal and stopwords in English
ENGLISH_STOPWORDS = set(stopwords.words('english'))
ENGLISH_WORDS = set(nltk.corpus.words.words())

# All About Data

## Data Import
Loading the data using Panda's `read_csv()` function.

In [ ]:
# Reading the data with 2 ways
# 1 - Reading within my Github Repo
txt_dat = pd.read_csv('../dataset/train.csv')

# 2 - Reading within the same folder
#txt_dat = pd.read_csv('train.csv')

## Data Exploration
Looking at the data from multiple perspectives.

Using `head()` and `tail()` methods to look at what columns there are within the dataframe, which may be useful and which are not.

In [ ]:
# Defining n rows
n = 5

# Calling and showing first and last n rows
display(txt_dat.head(n), txt_dat.tail(n), txt_dat.shape)

Looking at the data, we can see that we have about $159571$ entries.
We can also see that there are columns as such:
- `id`: Shows message id code from whatever platform this was scraped from. (Will be dropped)
- `comment_text`: main star of the show, text we have to process, it has upper-case letters, escape characters (`\n`,`\r`, etc.), and most likely special characters (non-latin-alphabet), URLs, IP addresses; removal needed.
- `toxic` (And its derivatives): Labelling the comment if it were a toxic or not.

## Defining What To Clean
In the above mentioned description of the data, there exists aspects that need removal, explanation will follow within this cell.

- Removal of id: Due to the nature of what this project is about, and usually a general case, IDs are not usually if at all used to gain insights from data, due to them being unique per entry and being nothing but an enumeration of entries.
- Cleaning within `comment_text`: Cleaning the comments will be separated into $x$ steps, namely:
    - Space and Tab removal: Removing of Newline Characters (`\n`) and Tabs (`\t`) will help remove special characters from text and avoiding noise within.
    - Uppercase: Due to standarisations within the NLP field, it has been agreed upon to change any uppercase letter into lowercase, to mitigate the face that words can be written using multiple permutations of the same letters but with different cases, so in order to for the machine to recognise the word and not have to account for $n^{52}$ different combinations for the word (Where $n$ is number of characters to represent a word, and $52$ is due to both upper and lower cases of a letter)
    - URL: Removal of URLs will prove beneficial, as it doesn't contribute much to the corpus nor is considered a baseline for labelling the comment.
    - IP Address: For security reasons.
    - Special Characters: Due to the non-existence of any in the Latin Alphabet which English uses, it would be useless to bother with them, although if this was a multi-lingual dataset, some characters from different languages would be needed to keep.
- Derivatives of `toxic`: For purposes of simplicity, I have decided to *"collapse"* the values that follow after the *`toxic`* column, as in summing the values into said column, then swapping values $>1$ to be just $1$ to indicate toxicity, implying that $0$ would be for non-toxic comments; as a result, this will water down into just a "Binary Classification" problem based on words.

In [ ]:
# Collapsing the column values onto toxic
txt_dat['toxic'] = txt_dat.iloc[:, 2:].sum(axis = 1)

# Drop the collapsed columns
# Along with the id column
txt_dat.drop(columns = ['id', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'], inplace = True)

In [ ]:
# Printing the head and describing the toxic column
display(txt_dat.head(n), txt_dat.describe())

As noted, due to collapsing the values onto `toxic`, values over 1 arose hence they will need handling in order to keep the labels as $1$ and $0$. A function doing just so will be implemented later.

### Regex Patterns
Identified Patterns that will be required to be used to capture specific instances of removable slices of text within the comments.

In [ ]:
# Newline, Tab spaces, etc.
newline_tabspace = r'[\r\n\t]'

# Match words starting with Uppercase letters
upper_words = r'([A-Z])\w+'

# Match Words that start with either Upper/lowercase letters
upperlower_words = r'[A-Za-z]\w+'

# Sub/Superscript characters
# Encountered previously
sub_sup_scripts = r'\w[²³¹⁰ⁱ⁴⁵⁶⁷⁸⁹⁺⁻⁼⁽⁾ⁿ]+'

# Punctuation
punc_pattern = r'[!\?.,\':;"]'

# Single Letters
single_letter = r'((?<=^)|(?<= )).((?=$)|(?= ))'

# Match URLs
url_pattern = r'(http|ftp|https):\/\/([\w+?\.\w+])+([a-zA-Z0-9\~\!\@\#\$\%\^\&\*\(\)_\-\=\+\\\/\?\.\:\;\'\,]*)?'

# Cleaning

## Functions and such
Functions to be used and applied on the dataframe.

Each function will be commented, whilst also writing an explanation to a grouped cell of functions indicating their use.

In [ ]:
def binarize(df: pd.DataFrame, colname: str):
    return np.where(df[colname] > 0, 1, 0)

def clean_comment(df: pd.DataFrame, colname: str):
    # Remove the newlines, tabs, etc.
    df[colname].replace(newline_tabspace, ' ', regex = True, inplace = True)

    # Remove URLs
    df[colname].replace(url_pattern, ' ', regex = True, inplace = True)

    # Remove superscript, subscript
    df[colname].replace(sub_sup_scripts, ' ', regex = True, inplace = True)

    # Remove punctuation
    df[colname].replace(punc_pattern, ' ', regex = True, inplace = True)

    # Remove Single Letters
    df[colname].replace(single_letter, ' ', regex = True, inplace = True)

    return df[colname]

A walkthrough the above functions:
- `binarize(df, colname)`: Function that takes a `pd.DataFrame`, `string` for input, dictating which DataFrame and Column within said DataFrame to carry out the operations on.
Due to the rise of values larger than 1 after collapsing the other column values, a requirement for binarizing (Chose to make the labels binary, for simplicity's sake) the values, $1$ for Toxic and $0$ for Non-Toxic, this is achieved by assigning any value larger than 0 to 1 otherwise stays as 0.

- `clean_comment(df, colname)`: Function that takes a `pd.DataFrame`, `string` for input, dictating which DataFrame and Column within said DataFrame to carry out the operations on.
The nature of the data given, is that it has a lot of *whitespaces*, *tabs*, *special characters*, *urls*, *punctuations*, *single letters*, which are all bound to be removed.
The function removes:
    - Newlines
    - Tabs
    - URLs
    - Sub/Super Scripts
    - Punctuation
    - Single Letters (From removal of some punctuation symbols)

Both functions return the updated column.

In [ ]:
txt_dat['comment_text'] = clean_comment(txt_dat, 'comment_text')

txt_dat['toxic'] = binarize(txt_dat, 'toxic')

display(txt_dat.head(n), txt_dat.tail(n), txt_dat.describe())

As we can see, there are no extra characters, like apostrophe's, extra punctuations, etc.

As well as having the maximum value to be 1 within the `toxic` column, although the mean is closer to 0 which suggest that the data is really imbalanced, which will require fixing later down the line, ideally it would be better to be closer to 0.5.

# NLP Preprocessing
Preprocessing the data with ways that are under the NLP PreProcessing Standards, as in:
- Tokenizing.
- Lowercasing.
- Removal of Stopwords.
- Removing Non-English words.
- Lemmatization/Stemming.

## Tokenization
Tokenization is the process of splitting up a sentence or a corpus into plain words (tokens) for variable reasons:
- To know which words are present.
- Count them.
- Et cetera.

In my implementation, I will be using the `regexp_tokenizer()` in order to tokenize the text within the `comment_text` column.

As well as keeping the tokens within the same dataframe as a list.

In [ ]:
# Function to tokenize a specific column within a passed DataFrame
def tokenize(dataframe: pd.DataFrame, colname: str):
    return [regexp_tokenize(row, upperlower_words) for row in dataframe[colname]]

In [ ]:
# Calling the tokenize() function on a new column within the main DataFrame
txt_dat['tokens'] = tokenize(txt_dat, 'comment_text')

# Checking if the function worked
txt_dat.head(n)

Notice how the sentences have been tokenised, each word is its own string, within a list of strings under the new `tokens` column.

## Lower Case Words
By conventional standards, changing the case of the words into lowercase has been agreed upon to negate the need to account for case sensitivity with the operations that follow to preprocess text.

In [ ]:
# function to lowercase text
def token_lower(token_list: list):
    return [token.lower() for token in token_list]

In [ ]:
# Iterating over the column
# Iterable is of type "list"
txt_dat['tokens'] = [token_lower(row) for row in txt_dat['tokens']]

# Displaying results of applying the above function
txt_dat.head()

Explaining how the function works first before observation.

Take the column `tokens`, it contains lists of tokens for each comment like: `[Explanation, Why, the, edits, made, under, my...]`.

Said list is what is being iterated on, which gets passed to the function, so for every iteration a new list of words gets passed to the function; but within the function, we iterate over each item within said list, so `'Explanantion` and `'Why'`, etc..

Therefore, passing over each token, we apply the `.lower()` function for the `str` data type; thus we get a lowercase version of the string.

After every iteration of lists, they get returned and reassigned where the original list used to be.

As for observation, we can see that the words did indeed become lowercase.

## Stopword Removal
Removing of stopwords drastically decreases the amount of words that need to be taken into consideration by the model, and contribute little to nothing regarding meaning.

Removing them would shave off redundant computations.

In [ ]:
# Function to remove the stopwords
def stopword_remover(wordlist: list):
    return [word for word in wordlist if word not in ENGLISH_STOPWORDS]

In [ ]:
# Displaying word count for first few lists
print([len(tokenlist) for tokenlist in txt_dat['tokens'].iloc[:5]])

# Iterating over column
# Iterable is the list of
# Lowercase english words
txt_dat['tokens'] = [stopword_remover(row) for row in txt_dat['tokens']]

print([len(tokenlist) for tokenlist in txt_dat['tokens'].iloc[:5]])

txt_dat.head(n)

As we can see by the decrease in the word count per token list, there was indeed a removal of redundant words.

As for how the function works, it is similar to how the previous ones work, passing the iterated list to iterate over the tokens within and apply a function; in our case it is a conditional to just include words that are not within the collection of words which are considered stopwords.

## Lemmatization & Stemming
Lemmatising/Stemming both have the same target in mind, to reach the root of the word but their difference is the algorithm used.

Stemming utilises a "Porter" algorithm which essentially just chops off common word endings, due to Stemmer using a crude old method, which is aimed for speed and efficiency, unlike lemmatizaton which morphologically analyses lexical changes in words to revert them back to their roots, unlike the chopping of "commonly found prefixes/suffixes" which stemming does.

In [ ]:
def lemmatize(wordlist: list):
    return [WordNetLemmatizer().lemmatize(token) for token in wordlist]

def stem(wordlist):
    return [SnowballStemmer("english").stem(token) for token in wordlist]

Essentially, after some look up, lemmatization is beneficial to reduce the word to its base, in a more human context and understanding, mainly used in chatbots, etc.

Whilest the stemmer is used to purely chop off the words' endings to get to the base, used more in sentiment analysis, which is our current case with the binary classification, hence it will be used after lemmatizer and english word remover are used.

## Non-English Words Removal
Due to the nature of the internet, it is bound that people don't use proper English words, but due to current knowledge of NLP process, it would be better to remove words that do not belong to the English language as a whole.

An important note to keep in mind is, calling this after lemmatization/stemming, due to `words` not having all forms of a singular word which may cause it to be removed.

In [ ]:
# Define Non-English word remover
def englishify(wordlist: list):
    return [word for word in wordlist if word in ENGLISH_WORDS]

In [ ]:
# Call the function
txt_dat['tokens'] = [englishify(row) for row in txt_dat['tokens']]

# Print some entries
txt_dat.head(n)

## Wrap Up NLP Preprocessing
In this section, a whole function will take in a column in order to apply ***all*** the above functions in one go.

Description and explanation for function order will be explained in either comments or a markdown cell following it.

In [ ]:
def preprocess(df: pd.DataFrame, colname: str):
    # List to hold the lists of tokens
    tokenlist = tokenize(df, colname)

    # Get number of tokens originally
    token_num = sum([len(t_list) for t_list in tokenlist])

    # Lowercase list of token lists
    # t_list -> (t)oken_list
    lower_tokens = [token_lower(t_list) for t_list in tokenlist]

    # Stopword-free list
    # tl_list -> (t)oken(l)lowercase_list
    stopwordless = [stopword_remover(tl_list) for tl_list in lower_tokens]

    # List of token lemmas
    # tls_list -> (t)oken(l)owercase(s)topwordless_list
    lemmas = [lemmatize(tls_list) for tls_list in stopwordless]

    # List of Lemmas that only exist in english
    # tlsl_list -> (t)oken(l)owercase(s)topwordless(l)lemma_list
    englishified = [englishify(tlsl_list) for tlsl_list in lemmas]

    # Get number of tokens after processing
    # (Englishify is the last function to remove)
    proc_num = sum([len(tlsle_list) for tlsle_list in englishified])

    # Reduction
    reduced_by = 1 - proc_num / token_num

    # Print counts and percentage
    print("Original Token Count : {}\nProcessed Token Count: {}\nReduction Percentage : {:.2%}".format(token_num, proc_num, reduced_by))

    # Return tuple of token list and processed tokens
    return (tokenlist, [stem(word) for word in englishified])

Explanation for how the wrap-up function works:
- Passing the DataFrame to be processed, along with the column name to be tokenized.
    - Each row within the column gets tokenized with use of the predefined RegEx expressions up above.
- Calculating the number of tokens to get a feel for how much we'll be dealing with.
- Applying a lowercase function to ever list within the list of lists assigned to the output of the tokenization function.
- Removing stopwords from the tokens.
- Lemmatizing the words, to reach the root of the word (Useful for removing non-English words later).
- Removal of Non-English words.
- Calculating the amount of tokens, `englishify()` is the last function that removes any tokens, afterwhich the reduction of tokens is calculated.
- Printing the calculated numbers.
- Returning a tuple of tokenised words and stemmed words.

In [ ]:
# Assigning output tuple to "new" column of tokens and new for processed
txt_dat['tokens'], txt_dat['processed'] = preprocess(txt_dat, 'comment_text')

# Printing to see progress
txt_dat.head(n)

# Transformer
Using PyTorch, a transformer will be implemented from scratch to classify comments into toxic or non-toxic.

Research Paper used as reference:
- [Attention Is All You Need](https://arxiv.org/pdf/1706.03762.pdf)

Website Articles that helped:
- [The Transformer Model](https://machinelearningmastery.com/the-transformer-model/)
- [Text Classifier with Pytorch Transformer](https://n8henrie.com/2021/08/writing-a-transformer-classifier-in-pytorch/)
- [Positional Encoding](https://theaisummer.com/positional-embeddings/)

In [ ]:
import math
import torch                                    # PyTorch
import torch.nn    as nn                        # PyTorch Neural Networks
import torch.optim as optim                     # Optimizers
from torchtext import data                      # Text Data Handling
from torch.nn  import functional as F           # Functions

## Fundamental Building Blocks

### Input Embedding
Similarly to other sequence transduction models, we use learned embeddings to convert the input
tokens and output tokens to vectors of dimension $d_{model}$. We also use the usual learned linear transformation and softmax function to convert the decoder output to predicted next-token probabilities. In
our model, we share the same weight matrix between the two embedding layers and the pre-softmax
linear transformation. In the embedding layers, we multiply those weights by $\sqrt{d_{model}}$.

In [ ]:
class InputEmbedding(nn.Module):
    def __init__(self, d_model: int, vocab_size: int):
        super().__init__()
        self.d_model = d_model
        self.voc_size = vocab_size
        self.embedding = nn.Embedding(vocab_size, d_model)

    def forward(self, input):
        return self.embedding(input) * (self.d_model ** 0.5)

### Positional Encoding
Since our model contains no recurrence and no convolution, in order for the model to make use of the
order of the sequence, we must inject some information about the relative or absolute position of the
tokens in the sequence. To this end, we add ***"positional encodings"*** to the input embeddings at the
bottoms of the encoder and decoder stacks. The positional encodings have the same dimension dmodel
as the embeddings, so that the two can be summed.

Thus, we will be working with the following trig functions of different frequencies:
$$ PE(pos, 2i) = sin(pos / 1000^{2\cdot i / d_{model}}) $$
$$ PE(pos, 2i + 1) = cos(pos / 1000^{2\cdot i / d_{model}}) $$

where $pos$ is the position and $i$ is the dimension. That is, each dimension of the positional encoding
corresponds to a sinusoid. The wavelengths form a geometric progression from $2\pi$ to $10000 \cdot 2\pi^{[1]}$. We
chose this function because we hypothesized it would allow the model to easily learn to attend by
relative positions, since for any fixed offset $k$, $PE_{pos+k}$ can be represented as a linear function of
$PE_{pos}$.

---

[1]: In this implementation, I will be using a logarithmic function instead of sinusoidal, due to finding that log tends to be more numerically stable.

In [ ]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model: int, seq_len: int, dropout: float):
        # https://pytorch.org/tutorials/beginner/transformer_tutorial.html
        super().__init__()
        self.d_model = d_model
        self.seq_len = seq_len
        self.dropout = nn.Dropout(dropout)

        # Create a matrix of shape (seq_len, d_model)
        pe = torch.zeros(seq_len, d_model)

        # Create a vector of shape (seq_len)
        numerator = torch.arange(0, seq_len, dtype=torch.float).unsqueeze(1) # (seq_len, 1)

        # Create a vector of shape (d_model)
        denominator = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model)) # (d_model / 2)

        # Apply sine to even indices
        pe[:, 0::2] = torch.sin(numerator * denominator)
        
        # Apply cosine to odd indices
        pe[:, 1::2] = torch.cos(numerator * denominator)

        # Add a batch dimension to the positional encoding
        pe = pe.unsqueeze(0) # (1, seq_len, d_model)
        
        # Register the positional encoding as a buffer
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:, :x.shape[1], :].requires_grad_(False) # (batch, seq_len, d_model)
        return self.dropout(x)

### Attention
The main star of the show and the key that emphasizes why the transformer is the better architecture.

Through remembering the prompts and contextually "understanding" the words whilst taking into consideration the meaning and value of the prior words.

Hence, the Multi-Head Attetntion formula is described by the following equation:
$$ MultiHeadAttention(Q, K, V) = Concat(head_1, \ldots, head_h)W^O$$

Where each $h_i$ is derived from the following formula:
$$ Attention(Q, K, V) = softmax(\frac{Q\cdot K^T}{\sqrt{d_{model}}})\cdot V$$

In this particular paper, the model has the hyperparameter of $h^{[1]}$ set to $8$, such that $d_k = d_v = d_{model} / h = 64$

Softmax:
$$ softmax(\vec{z})_{i} = \frac{e^{z_{i}}}{\sum^{n}_{j=1} e^{z_{j}}}$$
Where:
- $\vec{z} \rightarrow$ Input Vector.
- $e^{z_i} \rightarrow$ Standard Exponentiation for Input vector.
- $e^{z_j} \rightarrow$ Standard Exponentiation for Output Vector.
- $n \rightarrow$ number of classes in multi-class classifier.

---
[1]: h is the number of attention layers.

In [ ]:
class MultiHeadAttentionBlock(nn.Module):

    def __init__(self, d_model: int, h: int, dropout: float) -> None:
        super().__init__()
        self.d_model = d_model # Embedding vector size
        self.h = h # Number of heads

        self.d_k = d_model // h # Dimension of vector seen by each head
        self.w_q = nn.Linear(d_model, d_model, bias=False) # Wq
        self.w_k = nn.Linear(d_model, d_model, bias=False) # Wk
        self.w_v = nn.Linear(d_model, d_model, bias=False) # Wv
        self.w_o = nn.Linear(d_model, d_model, bias=False) # Wo
        self.dropout = nn.Dropout(dropout)

    @staticmethod
    def attention(query, key, value, mask, dropout: nn.Dropout):
        d_k = query.shape[-1]

        # Just apply the formula from the paper
        # (batch, h, seq_len, d_k) --> (batch, h, seq_len, seq_len)
        attention_scores = (query @ key.transpose(-2, -1)) / math.sqrt(d_k)

        if mask is not None:
            # Write a very low value (indicating -inf) to the positions where mask == 0
            attention_scores.masked_fill_(mask == 0, -1e9)
        attention_scores = attention_scores.softmax(dim=-1) # (batch, h, seq_len, seq_len) # Apply softmax

        if dropout is not None:
            attention_scores = dropout(attention_scores)
        # (batch, h, seq_len, seq_len) --> (batch, h, seq_len, d_k)
        # return attention scores which can be used for visualization
        return (attention_scores @ value), attention_scores

    def forward(self, q, k, v, mask):
        query = self.w_q(q) # (batch, seq_len, d_model) --> (batch, seq_len, d_model)
        key   = self.w_k(k) # (batch, seq_len, d_model) --> (batch, seq_len, d_model)
        value = self.w_v(v) # (batch, seq_len, d_model) --> (batch, seq_len, d_model)

        # (batch, seq_len, d_model) --> (batch, seq_len, h, d_k) --> (batch, h, seq_len, d_k)
        query = query.view(query.shape[0], query.shape[1], self.h, self.d_k).transpose(1, 2)

        key = key.view(key.shape[0], key.shape[1], self.h, self.d_k).transpose(1, 2)
        
        value = value.view(value.shape[0], value.shape[1], self.h, self.d_k).transpose(1, 2)

        # Calculate attention
        x, self.attention_scores = MultiHeadAttentionBlock.attention(query, key, value, mask, self.dropout)
        
        # Combine all the heads together
        # (batch, h, seq_len, d_k) --> (batch, seq_len, h, d_k) --> (batch, seq_len, d_model)
        x = x.transpose(1, 2).contiguous().view(x.shape[0], -1, self.h * self.d_k)

        # Multiply by Wo
        # (batch, seq_len, d_model) --> (batch, seq_len, d_model)  
        return self.w_o(x)

### Position-wise Feed Forward Network
In addition to attention sub-layers, each of the layers in our encoder and decoder contains a fully
connected feed-forward network, which is applied to each position separately and identically. This
consists of two linear transformations with a ReLU activation in between.

$$ FFN(x) = max(0, xW_{1} + b_{1})W_{2} + b_{2}$$

While the linear transformations are the same across different positions, they use different parameters
from layer to layer. Another way of describing this is as two convolutions with kernel size 1.
The dimensionality of input and output is $d_{model} = {512}$, and the inner-layer has dimensionality
$d_{ff} = {2048}$.

In [ ]:
class FeedForwardBlock(nn.Module):

    def __init__(self, d_model: int, d_ff: int, dropout: float) -> None:
        super().__init__()
        self.linear_1 = nn.Linear(d_model, d_ff) # w1 and b1
        self.dropout = nn.Dropout(dropout)
        self.linear_2 = nn.Linear(d_ff, d_model) # w2 and b2

    def forward(self, x):
        # (batch, seq_len, d_model) --> (batch, seq_len, d_ff) --> (batch, seq_len, d_model)
        return self.linear_2(self.dropout(torch.relu(self.linear_1(x))))

### Layer Normalization


In [ ]:
class LayerNormalization(nn.Module):

    def __init__(self, features: int, eps:float=10**-6) -> None:
        super().__init__()
        self.eps = eps
        self.alpha = nn.Parameter(torch.ones(features)) # alpha is a learnable parameter
        self.bias = nn.Parameter(torch.zeros(features)) # bias is a learnable parameter

    def forward(self, x):
        # x: (batch, seq_len, hidden_size)
         # Keep the dimension for broadcasting
        mean = x.mean(dim = -1, keepdim = True) # (batch, seq_len, 1)
        # Keep the dimension for broadcasting
        std = x.std(dim = -1, keepdim = True) # (batch, seq_len, 1)
        # eps is to prevent dividing by zero or when std is very small
        return self.alpha * (x - mean) / (std + self.eps) + self.bias

### Resuidual Connection
The skip layer between some MHA and FFN blocks

In [ ]:
class ResidualConnection(nn.Module):
    
        def __init__(self, features: int, dropout: float) -> None:
            super().__init__()
            self.dropout = nn.Dropout(dropout)
            self.norm = LayerNormalization(features)
    
        def forward(self, x, sublayer):
            return x + self.dropout(sublayer(self.norm(x)))

### Encoder Block
Consisting of $6^{[1]}$ layers of the same process. From the original design we can also see that there is a skip forward from:
- Before the MHA (Multi-Head Attention)
- Before the FFN (Feed Forward Network)

As well as consisting of $4$ blocks in total for each Encoder Block:
- Multi-Head Attention.
- First Normalization.
- Feed-Foward Network.
- Second Normalization.

It would be pointless if we just continuously passed the output from one block without transforming it in any way or shape, hence the introudction of the $LayerNorm(x + SubLayer(x))$ function, where $SubLayer(x)$ is a function produced by the layer itself to facilitate the residual connections, and produces an output of dimension $d_{model} = 512$.

---

[1]: Included in the original Paper, Section 3.1; Encoder Paragraph.

In [ ]:
# Single layer of encoder
class EncoderBlock(nn.Module):

    def __init__(self, features: int, self_attention_block: MultiHeadAttentionBlock, feed_forward_block: FeedForwardBlock, dropout: float) -> None:
        super().__init__()
        self.self_attention_block = self_attention_block
        self.feed_forward_block = feed_forward_block
        self.residual_connections = nn.ModuleList([ResidualConnection(features, dropout) for _ in range(2)])

    def forward(self, x, src_mask):
        x = self.residual_connections[0](x, lambda x: self.self_attention_block(x, x, x, src_mask))
        x = self.residual_connections[1](x, self.feed_forward_block)
        return x

# N Layers of encoders
class Encoder(nn.Module):

    def __init__(self, features: int, layers: nn.ModuleList) -> None:
        super().__init__()
        self.layers = layers
        self.norm = LayerNormalization(features)

    def forward(self, x, mask):
        for layer in self.layers:
            x = layer(x, mask)
        return self.norm(x)

### Projection

In [ ]:
class ProjectionLayer(nn.Module):

    def __init__(self, d_model, vocab_size) -> None:
        super().__init__()
        self.proj = nn.Linear(d_model, vocab_size)

    def forward(self, x) -> None:
        # (batch, seq_len, d_model) --> (batch, seq_len, vocab_size)
        return self.proj(x)

### Decoder Block
The decoder is also composed of a stack of $N = 6$ identical layers. In addition to the two
sub-layers in each encoder layer, the decoder inserts a third sub-layer, which performs multi-head attention over the output of the encoder stack. Similar to the encoder, we employ residual connections
around each of the sub-layers, followed by layer normalization. We also modify the self-attention
sub-layer in the decoder stack to prevent positions from attending to subsequent positions. This
masking, combined with fact that the output embeddings are offset by one position, ensures that the
predictions for position $i$ can depend only on the known outputs at positions less than $i$.

In [ ]:
# Single layer of Decoder
class DecoderBlock(nn.Module):

    def __init__(self, features: int, self_attention_block: MultiHeadAttentionBlock, cross_attention_block: MultiHeadAttentionBlock, feed_forward_block: FeedForwardBlock, dropout: float) -> None:
        super().__init__()
        self.self_attention_block = self_attention_block
        self.cross_attention_block = cross_attention_block
        self.feed_forward_block = feed_forward_block
        self.residual_connections = nn.ModuleList([ResidualConnection(features, dropout) for _ in range(3)])

    def forward(self, x, encoder_output, src_mask, tgt_mask):
        x = self.residual_connections[0](x, lambda x: self.self_attention_block(x, x, x, tgt_mask))
        x = self.residual_connections[1](x, lambda x: self.cross_attention_block(x, encoder_output, encoder_output, src_mask))
        x = self.residual_connections[2](x, self.feed_forward_block)
        return x

# N layers of Decoders
class Decoder(nn.Module):

    def __init__(self, features: int, layers: nn.ModuleList) -> None:
        super().__init__()
        self.layers = layers
        self.norm = LayerNormalization(features)

    def forward(self, x, encoder_output, src_mask, tgt_mask):
        for layer in self.layers:
            x = layer(x, encoder_output, src_mask, tgt_mask)
        return self.norm(x)

## Crescendo
Bringing all the building blocks and pieces to shape together **The Transformer**.

In [ ]:
class Transformer(nn.Module):

    def __init__(self, encoder: Encoder, src_embed: InputEmbedding, src_pos: PositionalEncoding, projection_layer: ProjectionLayer) -> None:
        super().__init__()
        self.encoder = encoder
        self.src_embed = src_embed
        self.src_pos = src_pos
        self.projection_layer = projection_layer

    def encode(self, src, src_mask):
        # (batch, seq_len, d_model)
        src = self.src_embed(src)
        src = self.src_pos(src)
        return self.encoder(src, src_mask)
    
    def project(self, x):
        # (batch, seq_len, vocab_size)
        return self.projection_layer(x)
    
def build_transformer(src_vocab_size: int, src_seq_len: int, d_model: int=512, N: int=6, h: int=8, dropout: float=0.1, d_ff: int=2048) -> Transformer:
    # Create the embedding layers
    src_embed = InputEmbedding(d_model, src_vocab_size)

    # Create the positional encoding layers
    src_pos = PositionalEncoding(d_model, src_seq_len, dropout)
    
    # Create the encoder blocks
    encoder_blocks = []
    for _ in range(N):
        encoder_self_attention_block = MultiHeadAttentionBlock(d_model, h, dropout)
        feed_forward_block = FeedForwardBlock(d_model, d_ff, dropout)
        encoder_block = EncoderBlock(d_model, encoder_self_attention_block, feed_forward_block, dropout)
        encoder_blocks.append(encoder_block)

    # Create the encoder
    encoder = Encoder(d_model, nn.ModuleList(encoder_blocks))
    
    # Create the projection layer
    projection_layer = ProjectionLayer(d_model, src_vocab_size)
    
    # Create the transformer
    transformer = Transformer(encoder, src_embed, src_pos, projection_layer)
    
    # Initialize the parameters
    for p in transformer.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform_(p)
    
    return transformer

# Bringing it All Together
Dividing Data and turning it into pytorch tensor compatible input.

Input into the transformer implemented above and evaluating.